# 1.0 Import Dependencies

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

import datetime as dt

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

# 2.0 Load Data

In [2]:
df = pd.read_csv(r'data_preprocessing_1.csv')

In [3]:
df.head()

,link,likes,comment_counts,dates,captions,type_posts,username,following,post,followers,year,month,day,hour,year_month,likes_engagement,comments_engagement,total_engagement,len_capt,len_capt_wo_punct,len_capt_punct,n_words,avg_char_words,n_numeric
0,https://www.instagram.com/p/CCDUy0rhY4U/,43953,5,2020-06-30-15,"from our honeymoon,summer 2019 !______________...",GraphVideo,princessyahrini,649,4769,33500000,2020,6,30,15,202006,0.131203,0.000015,0.131218,81,78,3,6,0.074074,1
1,https://www.instagram.com/p/CB4WqVRBAOP/,145632,0,2020-06-26-08,•selamat pagi !__________________ 𝓢𝓨𝓡_________...,GraphImage,princessyahrini,649,4769,33500000,2020,6,26,8,202006,0.434722,0.000000,0.434722,72,69,3,4,0.055556,0
2,https://www.instagram.com/p/CB0IFkKh0Py/,79662,65,2020-06-24-17,“ i have nothing “_________________ 𝓢𝓨𝓡_______...,GraphVideo,princessyahrini,649,4769,33500000,2020,6,24,17,202006,0.237797,0.000194,0.237991,71,68,3,6,0.084507,0
3,https://www.instagram.com/p/CBzudkKBc5v/,76461,0,2020-06-24-13,•a day well spent with my beautiful soul siste...,GraphImage,princessyahrini,649,4769,33500000,2020,6,24,13,202006,0.228242,0.000000,0.228242,230,219,11,23,0.100000,0
4,https://www.instagram.com/p/CBvH-ZaB5c5/,141927,0,2020-06-22-18,"•alhamdulillah,amin allahuma amin ... amin ya ...",GraphVideo,princessyahrini,649,4769,33500000,2020,6,22,18,202006,0.423663,0.000000,0.423663,115,108,7,9,0.078261,0


# 3.0 Create Base Table

In [4]:
base_table = df.groupby(['username', 'year_month']).agg({
    'link' : 'count'
}).reset_index().rename(columns={
    'link' : 'n_post'
})

In [5]:
#create features lag of n_post (last 3 month)
#number of n_post 1 months ago
base_table['n_post_01'] = base_table.groupby(['username'])['n_post'].shift(1).fillna(0)

#number of n_post 2 months ago
base_table['n_post_02'] = base_table.groupby(['username'])['n_post'].shift(2).fillna(0)

#number of n_post 3 months ago
base_table['n_post_03'] = base_table.groupby(['username'])['n_post'].shift(3).fillna(0)

## 3.1 Features type_posts

In [6]:
df_type_post = df.groupby(['username', 'year_month'])['type_posts'].value_counts().unstack(2).reset_index().fillna(0)

In [7]:
base_table['n_img_post'] = df_type_post['GraphImage']
base_table['n_vid_post'] = df_type_post['GraphVideo']
base_table['n_sidecar_post'] = df_type_post['GraphSidecar']

## 3.2 Features Likes

In [8]:
#create features sum, avg and stdev of likes
base_table['sum_likes'] = df.groupby(['username', 'year_month'])['likes'].sum().reset_index()['likes']
base_table['avg_likes'] = df.groupby(['username', 'year_month'])['likes'].mean().reset_index()['likes']
base_table['std_likes'] = df.groupby(['username', 'year_month'])['likes'].std().reset_index()['likes'].fillna(0)

In [9]:
#create features lag of sum_likes and mean_likes (last 3 month)
base_table['avg_likes_01'] = base_table.groupby(['username'])['avg_likes'].shift(1).fillna(0)
base_table['avg_likes_02'] = base_table.groupby(['username'])['avg_likes'].shift(2).fillna(0)
base_table['avg_likes_03'] = base_table.groupby(['username'])['avg_likes'].shift(3).fillna(0)

base_table['sum_likes_01'] = base_table.groupby(['username'])['sum_likes'].shift(1).fillna(0)
base_table['sum_likes_02'] = base_table.groupby(['username'])['sum_likes'].shift(2).fillna(0)
base_table['sum_likes_03'] = base_table.groupby(['username'])['sum_likes'].shift(3).fillna(0)

## 3.3 Features Comments

In [10]:
#create features sum, avg and stdev of comments
base_table['sum_comments'] = df.groupby(['username', 'year_month'])['comment_counts'].sum().reset_index()['comment_counts']
base_table['avg_comments'] = df.groupby(['username', 'year_month'])['comment_counts'].mean().reset_index()['comment_counts']
base_table['std_comments'] = df.groupby(['username', 'year_month'])['comment_counts'].std().reset_index()['comment_counts'].fillna(0)

In [11]:
#create features lag of sum_comments and mean_comments (last 3 month)
base_table['avg_comments_01'] = base_table.groupby(['username'])['avg_comments'].shift(1).fillna(0)
base_table['avg_comments_02'] = base_table.groupby(['username'])['avg_comments'].shift(2).fillna(0)
base_table['avg_comments_03'] = base_table.groupby(['username'])['avg_comments'].shift(3).fillna(0)

base_table['sum_comments_01'] = base_table.groupby(['username'])['sum_comments'].shift(1).fillna(0)
base_table['sum_comments_02'] = base_table.groupby(['username'])['sum_comments'].shift(2).fillna(0)
base_table['sum_comments_03'] = base_table.groupby(['username'])['sum_comments'].shift(3).fillna(0)

## 3.4 Features Captions

In [16]:
#create features sum, avg and stdev of captions_wo_punct
base_table['sum_capt_wo_punct'] = df.groupby(['username', 'year_month'])['len_capt_wo_punct'].sum().reset_index()['len_capt_wo_punct']
base_table['avg_capt_wo_punct'] = df.groupby(['username', 'year_month'])['len_capt_wo_punct'].mean().reset_index()['len_capt_wo_punct']
base_table['std_capt_wo_punct'] = df.groupby(['username', 'year_month'])['len_capt_wo_punct'].std().reset_index()['len_capt_wo_punct'].fillna(0)

#create features sum, avg and stdev of captions_w_punct
base_table['sum_capt_w_punct'] = df.groupby(['username', 'year_month'])['len_capt_punct'].sum().reset_index()['len_capt_punct']
base_table['avg_capt_w_punct'] = df.groupby(['username', 'year_month'])['len_capt_punct'].mean().reset_index()['len_capt_punct']
base_table['std_capt_w_punct'] = df.groupby(['username', 'year_month'])['len_capt_punct'].std().reset_index()['len_capt_punct'].fillna(0)

#create features sum, avg and stdev of n_words
base_table['sum_n_words'] = df.groupby(['username', 'year_month'])['n_words'].sum().reset_index()['n_words']
base_table['avg_n_words'] = df.groupby(['username', 'year_month'])['n_words'].mean().reset_index()['n_words']
base_table['std_n_words'] = df.groupby(['username', 'year_month'])['n_words'].std().reset_index()['n_words'].fillna(0)

#create features sum, avg and stdev of avg_char_words
base_table['sum_avg_char_words'] = df.groupby(['username', 'year_month'])['avg_char_words'].sum().reset_index()['avg_char_words']
base_table['avg_avg_char_words'] = df.groupby(['username', 'year_month'])['avg_char_words'].mean().reset_index()['avg_char_words']
base_table['std_avg_char_words'] = df.groupby(['username', 'year_month'])['avg_char_words'].std().reset_index()['avg_char_words'].fillna(0)

In [18]:
#create features lag of captions (last 3 month)
base_table['sum_capt_wo_punct_01'] = base_table.groupby(['username'])['sum_capt_wo_punct'].shift(1).fillna(0)
base_table['sum_capt_wo_punct_02'] = base_table.groupby(['username'])['sum_capt_wo_punct'].shift(2).fillna(0)
base_table['sum_capt_wo_punct_03'] = base_table.groupby(['username'])['sum_capt_wo_punct'].shift(3).fillna(0)

base_table['sum_capt_w_punct_01'] = base_table.groupby(['username'])['sum_capt_w_punct'].shift(1).fillna(0)
base_table['sum_capt_w_punct_02'] = base_table.groupby(['username'])['sum_capt_w_punct'].shift(2).fillna(0)
base_table['sum_capt_w_punct_03'] = base_table.groupby(['username'])['sum_capt_w_punct'].shift(3).fillna(0)

base_table['sum_n_words_01'] = base_table.groupby(['username'])['sum_n_words'].shift(1).fillna(0)
base_table['sum_n_words_02'] = base_table.groupby(['username'])['sum_n_words'].shift(2).fillna(0)
base_table['sum_n_words_03'] = base_table.groupby(['username'])['sum_n_words'].shift(3).fillna(0)

base_table['sum_avg_char_words_01'] = base_table.groupby(['username'])['sum_avg_char_words'].shift(1).fillna(0)
base_table['sum_avg_char_words_02'] = base_table.groupby(['username'])['sum_avg_char_words'].shift(2).fillna(0)
base_table['sum_avg_char_words_03'] = base_table.groupby(['username'])['sum_avg_char_words'].shift(3).fillna(0)

In [20]:
base_table['sum_n_numeric'] = df.groupby(['username', 'year_month'])['n_numeric'].sum().reset_index()['n_numeric']

## 3.5 Features Engagement

In [28]:
#create features sum and avg engagement
base_table['sum_likes_engagement'] = df.groupby(['username', 'year_month'])['likes_engagement'].sum().reset_index()['likes_engagement']
base_table['sum_comments_engagement'] = df.groupby(['username', 'year_month'])['comments_engagement'].sum().reset_index()['comments_engagement']
base_table['sum_total_engagement'] = df.groupby(['username', 'year_month'])['total_engagement'].sum().reset_index()['total_engagement']

base_table['avg_likes_engagement'] = df.groupby(['username', 'year_month'])['likes_engagement'].mean().reset_index()['likes_engagement']
base_table['avg_comments_engagement'] = df.groupby(['username', 'year_month'])['comments_engagement'].mean().reset_index()['comments_engagement']
base_table['avg_total_engagement'] = df.groupby(['username', 'year_month'])['total_engagement'].mean().reset_index()['total_engagement']

In [30]:
#create features lag of engagement (last 3 month)
base_table['avg_total_engagement_01'] = base_table.groupby(['username'])['avg_total_engagement'].shift(1).fillna(0)
base_table['avg_total_engagement_02'] = base_table.groupby(['username'])['avg_total_engagement'].shift(2).fillna(0)
base_table['avg_total_engagement_03'] = base_table.groupby(['username'])['avg_total_engagement'].shift(3).fillna(0)

## 3.6 Target Variable

In [31]:
#Create our target Variable
base_table['y_month_01'] = base_table.groupby(['username'])['avg_total_engagement'].shift(-1).fillna(0)

In [32]:
base_table

,username,year_month,n_post,n_post_01,n_post_02,n_post_03,n_img_post,n_vid_post,n_sidecar_post,sum_likes,avg_likes,std_likes,avg_likes_01,avg_likes_02,avg_likes_03,sum_likes_01,sum_likes_02,sum_likes_03,sum_comments,avg_comments,std_comments,avg_comments_01,avg_comments_02,avg_comments_03,sum_comments_01,sum_comments_02,sum_comments_03,sum_capt_wo_punct,avg_capt_wo_punct,std_capt_wo_punct,sum_capt_w_punct,avg_capt_w_punct,std_capt_w_punct,sum_n_words,avg_n_words,std_n_words,sum_avg_char_words,avg_avg_char_words,std_avg_char_words,sum_capt_wo_punct_01,sum_capt_wo_punct_02,sum_capt_wo_punct_03,sum_capt_w_punct_01,sum_capt_w_punct_02,sum_capt_w_punct_03,sum_n_words_01,sum_n_words_02,sum_n_words_03,sum_avg_char_words_01,sum_avg_char_words_02,sum_avg_char_words_03,sum_n_numeric,sum_likes_engagement,sum_comments_engagement,sum_total_engagement,avg_likes_engagement,avg_comments_engagement,avg_total_engagement,avg_total_engagement_01,avg_total_engagement_02,avg_total_engagement_03,y_month_01
0,1milliondance,201906,1,0.0,0.0,0.0,0.0,0.0,1.0,29763,29763.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,47,47.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,59,59.000000,0.000000,3,3.000000,0.000000,8,8.000000,0.000000,0.129032,0.129032,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0,2.289462,0.003615,2.293077,2.289462,0.003615,2.293077,0.000000,0.000000,0.000000,1.793402
1,1milliondance,201907,9,1.0,0.0,0.0,6.0,2.0,1.0,209598,23288.666667,15705.132250,29763.000000,0.000000,0.000000,29763.0,0.0,0.0,230,25.555556,14.552587,47.000000,0.000000,0.000000,47.0,0.0,0.0,724,80.444444,33.211862,47,5.222222,2.108185,114,12.666667,7.106335,1.270923,0.141214,0.028321,59.0,0.0,0.0,3.0,0.0,0.0,8.0,0.0,0.0,0.129032,0.000000,0.000000,0,16.122923,0.017692,16.140615,1.791436,0.001966,1.793402,2.293077,0.000000,0.000000,1.354944
2,1milliondance,201908,11,9.0,1.0,0.0,5.0,4.0,2.0,193379,17579.909091,6093.849825,23288.666667,29763.000000,0.000000,209598.0,29763.0,0.0,378,34.363636,27.292023,25.555556,47.000000,0.000000,230.0,47.0,0.0,907,82.454545,56.196732,74,6.727273,7.862454,149,13.545455,8.594925,1.677293,0.152481,0.029048,724.0,59.0,0.0,47.0,3.0,0.0,114.0,8.0,0.0,1.270923,0.129032,0.000000,2,14.875308,0.029077,14.904385,1.352301,0.002643,1.354944,1.793402,2.293077,0.000000,1.610500
3,1milliondance,201909,2,11.0,9.0,1.0,0.0,1.0,1.0,41838,20919.000000,15655.344135,17579.909091,23288.666667,29763.000000,193379.0,209598.0,29763.0,35,17.500000,21.920310,34.363636,25.555556,47.000000,378.0,230.0,47.0,51,25.500000,6.363961,5,2.500000,0.707107,5,2.500000,0.707107,0.177866,0.088933,0.002795,907.0,724.0,59.0,74.0,47.0,3.0,149.0,114.0,8.0,1.677293,1.270923,0.129032,0,3.218308,0.002692,3.221000,1.609154,0.001346,1.610500,1.354944,1.793402,2.293077,1.384108
4,1milliondance,201910,5,2.0,11.0,9.0,3.0,0.0,2.0,89894,17978.800000,2504.253522,20919.000000,17579.909091,23288.666667,41838.0,193379.0,209598.0,73,14.600000,5.549775,17.500000,34.363636,25.555556,35.0,378.0,230.0,498,99.600000,30.533588,36,7.200000,2.489980,73,14.600000,4.335897,0.683875,0.136775,0.000317,51.0,907.0,724.0,5.0,74.0,47.0,5.0,149.0,114.0,0.177866,1.677293,1.270923,1,6.914923,0.005615,6.920538,1.382985,0.001123,1.384108,1.610500,1.354944,1.793402,1.012781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4355,zoeabbasjackson,202002,12,4.0,5.0,5.0,4.0,4.0,4.0,1262483,105206.916667,48387.863100,76610.250000,128589.000000,105608.400000,306441.0,642945.0,528042.0,5329,444.083333,487.669310,105.250000,536.400000,213.600000,421.0,2682.0,1068.0,908,75.666667,80.601526,87,7.250000,7.111131,173,14.416667,15.808274,2.073692,0.172808,0.025087,754.0,110.0,178.0,27.0,18.0,17.0,118.0,19.0,33.0,0.640979,0.851883,0.994161,4,128.995913,0.544498,129.540411,10.749659,0.045375,10.79503

In [33]:
base_table.to_csv('data_feature_engineering.csv', index=False)